In [13]:
import sys, os

NLP_PATH = '/'.join(os.path.abspath('.').split('/')[:-1]) + '/'
sys.path.append(NLP_PATH)
from nlp.text import extractor as xt
from nlp.models.message_classification import SimpleClassifier
from nlp.utils.model_output_management import OutputHelper
from nlp.models.similarity_calculation import MessageSimilarity
from nlp.models.summarization import TFIDF
from nlp.grammar import tokenizer as nt
from nlp.viz.cloud import Wordcloud
from nlp.viz import graph as vg

casdb = xt.CassandraExtractor(cluster_ips=['54.175.189.47'],
                              session_keyspace='test_keyspace',
                              table_name='awaybot_messages')
channels = ['general']
msg_sim = MessageSimilarity()
FONT_PATH = '../nlp/data/font/Ranga-Regular.ttf'
IMG_FOLDER = '../nlp/data/img/'
viz_topics = 0
for channel in channels:
    for p in xrange(1, 7,1):
        min_topic_length = 5 if p < 2 else 10
        msg_stream = casdb.get_messages(type_of_query='week', periods=p, channel=channel, min_words=5)
        classifier = SimpleClassifier(message_similarity=msg_sim)
        classified_window = classifier.classify_stream(
            msg_stream, low_threshold=.4, high_threshold=.7, low_step=.05, high_step=.02,
            max_messages=10000, verbose=False)
        if len(classified_window) == 0:
            continue
        n_model = TFIDF(classified_window, cleaner=nt.SimpleCleaner(), n_grams=5)
        for t, topic in enumerate(classified_window):
            if len(topic) >= min_topic_length:
                viz_topics += 1
                viz = vg.Wordgraph(n_model, document_id=t, max_words=8)
                viz_path = IMG_FOLDER + 'graph_topic_{}_{}_{}_{}.png'.format(channel, 'weeks', p, viz_topics)
                viz.save_png(viz_path)

 -- Loading GloVe, this might take a few (10~30) seconds... -- 

... Done, processed 45 messages
... Done, processed 185 messages
... Done, processed 185 messages
... Done, processed 196 messages
... Done, processed 259 messages
... Done, processed 310 messages


In [1]:
import sys, os

NLP_PATH = '/'.join(os.path.abspath('.').split('/')[:-1]) + '/'
sys.path.append(NLP_PATH)
from nlp.text import extractor as xt
from nlp.models.message_classification import SimpleClassifier
from nlp.utils.model_output_management import OutputHelper
from nlp.models.similarity_calculation import MessageSimilarity
from nlp.models.summarization import TFIDF
from nlp.grammar import tokenizer as nt
from nlp.viz.cloud import Wordcloud
from nlp.viz import graph as vg
team = 'Slack-pack'
channel = 'general'


casdb = xt.CassandraExtractor(cluster_ips=['54.175.189.47'],
                              session_keyspace='test_keyspace',
                              table_name='awaybot_messages')
channels = ['general']
msg_sim = MessageSimilarity()
FONT_PATH = '../nlp/data/font/Ranga-Regular.ttf'
IMG_FOLDER = '../nlp/data/img/'
viz_topics = 0
for p in xrange(2,4):
    msg_stream = casdb.get_messages(type_of_query='week', periods=p, channel=channel, min_words=5)
    classifier = SimpleClassifier(message_similarity=msg_sim)
    classified_window = classifier.classify_stream(msg_stream, low_threshold=.4, high_threshold=.7, low_step=.05, high_step=.02, max_messages=10000, verbose=False)
    image_loader = OutputHelper()
    if len(classified_window) == 0:
        image_loader.updateImageCount(team, channel, p, 'weeks')

    else:
        # Create a model using the corpus
        uni_model = TFIDF(window=classified_window, cleaner=nt.SimpleCleaner(), n_grams=4)
        # bigram_model = Model(window=classified_window, cleaner=nt.SimpleCleaner(), n_grams=2)  # if we wanted a bigram model

        viz_topics = 0
        for t, topic in enumerate(classified_window):  # one(?) per topic
            if len(topic) >= 10:
                viz = vg.Wordgraph(model=uni_model, document_id=t, max_words=4)
                viz_path = IMG_FOLDER + 'graph_topic_{}_{}_{}_{}.png'.format(channel, 'weeks', p, viz_topics)
                viz.save_png(viz_path)
                viz_topics += 1

                # Call image_loader with: img_path + starter_message_url + team + channel + duration + duration_unit
                image_loader.add_viz(viz_path, topic.start_message.url, topic.start_message.team, channel, p, 'weeks')
                # TODO: parameter-ize the duration unit
    #     if viz_topics:
    #         image_loader.upload()
    #     else:
    #         # Create a sdb item that records '0' images for that particular channel and duration
    #         image_loader.updateImageCount(team, channel, p, 'weeks')
        # logger.info(
        #     'Updloaded {} Images for channel {} and '
        #     'duration: {} {}'.format(viz_topics, channel, p, 'week(s)'))

 -- Loading GloVe, this might take a few (10~30) seconds... -- 

... Done, processed 185 messages
... Done, processed 185 messages
